### **PydanticOuputParser**

`PydanticOuputParser`는 모델의 출력을 더 구조화된 정보로 변환 

- 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공 

주요 메서드 

- `get_format_instructions()`: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 제공. 
    - 모델이 출력해야 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환 

- `parse()`: 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환. 
    - Pydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환.


#### 참고 자료
- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)

In [5]:
from langchain_groq import ChatGroq 
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatGroq(model = 'gemma2-9b-it')



In [15]:
# 이메일 예시 문장
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


### **출력 파서를 사용하지 않을때**

In [18]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template('''입력된 텍스트는 이메일 입니다 이메일에서 중요한 내용을 추출해주세요 
                                      
                                      {email_conversation}''')

chain = prompt | llm

response = chain.invoke(email_conversation)

print(response.content)

## 이메일 중요 내용 요약

**1. 발신자:** 김철수 상무 (바이크코퍼레이션)

**2. 수신자:** 이은채 대리 (Teddy International)

**3. 제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

**4. 내용:**

* 바이크코퍼레이션은 "ZENESIS" 자전거에 대한 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보를 필요로 함.
* 협력 가능성을 논의하기 위해 1월 15일(화) 오전 10시, Teddy International 사무실에서 미팅을 제안.


**5. 요청사항:**

* ZENESIS 모델 상세 브로슈어 제공
* 1월 15일 미팅 약속






### **Pydantic 사용**

In [ ]:
class EmailSummary(BaseModel):
    sender: str = Field(description="보낸 사람")
    mail : str = Field(description="보낸 사람 이메일 주소")
    receiver: str = Field(description="받는 사람")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일에 언급된 미팅 날짜와 시간")
    
    
# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender": {"description": "보낸 사람", "title": "Sender", "type": "string"}, "mail": {"description": "보낸 사람 이메일 주소", "title": "Mail", "type": "string"}, "receiver": {"description": "받는 사람", "title": "Receiver", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["sender", "mail", "receiver", "subject", "summary", "

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)